In [4]:
import os 
import pandas as pd
    
def create_plateau(n, df):
    plateau_counts = []
    max_val = float(df.max())
    print("Max: ",max_val)
    min_val = float(df.min())
    print("Min: ",min_val)
    # max_val = 1157.6
    # min_val = 43.2
    plateau_list = []
    plateau_size = (max_val-min_val)/n
    print("Plateua Size: ",plateau_size)
    current_plateau = min_val
    for i in range(0,n):
        new_plateau = current_plateau + plateau_size
        print(str(current_plateau) + " --- " + str(new_plateau))
        df_filter = int(df[(df["BUS"] > current_plateau) & (df["BUS"] <= new_plateau)].count()[0])
        current_plateau = new_plateau
        print(df_filter)
        plateau_counts.append(df_filter)
    return plateau_counts

def only_max(df, tolerance):
    max_val = float(df.max())
    plateau_count = df[df["BUS"] >= (max_val-tolerance)].count()[0]
    return plateau_count


def load_data(path, plateau_size=None):
    no_dash_list = []
    dash_list = []

    for dir in os.listdir(path):
        for subdir in os.listdir(os.path.join(path,dir)):
            load_path = os.path.join(path, dir, subdir)
            df = pd.read_csv(load_path)
            df = df[["BUS"]]
            if plateau_size is None:
                plateau_counts = only_max(df,tolerance=10)
            else:
                plateau_counts = create_plateau(plateau_size, df)
            if "no" in subdir:
                no_dash_list.append(plateau_counts)
            else:
                dash_list.append(plateau_counts)
    NoDash = pd.DataFrame(no_dash_list)
    NoDash["Dash"] = "no"
    Dash = pd.DataFrame(dash_list)
    Dash["Dash"] = "yes"
    df = pd.concat((NoDash,Dash))
    return df.sample(frac=1).reset_index(drop=True)
    

In [5]:
# if platue_size is not assigned like: load_data(path = "Printer3D")
# then there are no plateaus. Instead there is a tolerance of the maximum value of the current
# and everything above this tolerance is counted 
# Example: max value of current: 1000mA
# Tolerance: 100mA
# Result: Every current above 900mA is counted

# df = load_data(path = "Printer3D")

# with plateaus: assign desired number of plateaus
df = load_data(path = "Printer3D", plateau_size=20)
df

Max:  1150.5
Min:  84.5
Plateua Size:  53.3
84.5 --- 137.8
12819
137.8 --- 191.10000000000002
14082
191.10000000000002 --- 244.40000000000003
28856
244.40000000000003 --- 297.70000000000005
18410
297.70000000000005 --- 351.00000000000006
64548
351.00000000000006 --- 404.30000000000007
4
404.30000000000007 --- 457.6000000000001
2
457.6000000000001 --- 510.9000000000001
4
510.9000000000001 --- 564.2
12
564.2 --- 617.5
0
617.5 --- 670.8
2
670.8 --- 724.0999999999999
1
724.0999999999999 --- 777.3999999999999
3
777.3999999999999 --- 830.6999999999998
0
830.6999999999998 --- 883.9999999999998
1
883.9999999999998 --- 937.2999999999997
3018
937.2999999999997 --- 990.5999999999997
5214
990.5999999999997 --- 1043.8999999999996
22907
1043.8999999999996 --- 1097.1999999999996
9927
1097.1999999999996 --- 1150.4999999999995
13346
Max:  1144.0
Min:  51.5
Plateua Size:  54.625
51.5 --- 106.125
3110
106.125 --- 160.75
17552
160.75 --- 215.375
16911
215.375 --- 270.0
34502
270.0 --- 324.625
49107
324.62

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,Dash
0,9502,14498,17538,34322,74011,46,4,9,11,0,...,0,1,2,20,5280,5185,9390,9809,13956,no
1,9310,16232,18012,34285,71027,17,8,5,12,1,...,1,1,3,402,7337,3589,12091,11060,13547,no
2,12819,14082,28856,18410,64548,4,2,4,12,0,...,1,3,0,1,3018,5214,22907,9927,13346,yes
3,7573,15486,17241,35345,75614,291,4,8,12,3,...,0,1,1,147,5890,4715,9916,10740,13935,yes
4,1383,15159,16531,37660,34431,44164,0,14,17,0,...,0,0,0,32,5629,5529,6639,13550,14212,no
5,3110,17552,16911,34502,49107,28286,0,9,10,0,...,0,0,0,46,1392,5091,6169,13096,15952,yes


In [6]:
# store dataset for train/test model
df.to_csv("3d_printer_plateau.csv", index=False)

In [9]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import cross_val_score 
y = df[["Dash"]]
X = df.drop("Dash", axis=1)
dt = DecisionTreeClassifier()
# don't want to extract testset, instead using Cross Validation
cv_results = cross_val_score(dt, X, y, cv=3)
print(cv_results)

[0.  0.5 0.5]


Clean data_dash0 because of late start

In [17]:
df_1 = pd.read_csv("Printer3D/Dash/data_dash1.csv")
# compare size with another dataset
print(df_1.shape[0])
df = pd.read_csv("data_dash0.csv")
# delete the starting procedure
df = df.loc[40000:]
# store it without index
df.to_csv("Printer3D/Dash/data_dash0_gereinigt.csv", index=False)
# validate size of new datas
df


191235


,Time,MCU,BUS
40000,120324.0,5.5,1022.9
40001,120327.0,5.5,1029.9
40002,120330.0,5.6,1032.6
40003,120333.0,6.0,1033.1
40004,120336.0,5.2,1028.5
...,...,...,...
233155,695864.0,5.8,115.4
233156,695867.0,5.8,117.0
233157,695870.0,6.3,115.8
233158,695873.0,5.8,116.2
